In [36]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [73]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [38]:
from implementations import *

## Do your thing crazy machine learning thing here :) ...

### Testing functions

In [39]:
#GD
initial_w=np.zeros(tX.shape[1])
max_iters=20
gamma=1e-9
w,loss = least_squares_GD(y,tX,initial_w,max_iters,gamma)
print(f'w:{w}\nloss:{loss}')

w:[ 2.11810865e-06 -5.29191651e-07 -4.97156569e-07 -1.22343323e-07
  5.37356098e-06  5.86712110e-06  5.36618437e-06 -1.39858758e-08
 -1.21122986e-07 -6.30640693e-07 -1.17564129e-08  6.81002513e-09
  5.37217447e-06 -1.32872186e-07  4.23758270e-11 -1.04172239e-10
 -2.93867921e-07  1.49861965e-10 -1.18664880e-10 -2.38180362e-07
  3.15398991e-10 -9.49755057e-07 -3.58441808e-09  3.60115394e-06
  3.71824597e-06  3.71825488e-06  5.33200096e-06  5.37177045e-06
  5.37168402e-06 -2.03900561e-07]
loss:0.4878776005124261


In [40]:
#SDG
initial_w=np.zeros(tX.shape[1])
max_iters=20
gamma=1e-9
w,loss = least_squares_SGD(y,tX,initial_w,max_iters,gamma)
print(f'w:{w}\nloss:{loss}')

w:[ 2.29854837e-06 -5.80259034e-07 -6.57141257e-07 -2.32002403e-07
  8.36615248e-06  9.87353409e-06  8.34913317e-06 -2.55303945e-08
 -1.19371222e-07 -7.25928236e-07 -1.75217610e-08  1.11238915e-08
  8.36363786e-06 -1.91483477e-07 -4.72267512e-09 -1.69780606e-08
 -3.56531060e-07  8.00199135e-09 -2.53045907e-09 -3.68395915e-07
  1.94144261e-09 -1.19969000e-06 -2.78592438e-09  8.50881123e-06
  8.61615414e-06  8.62012069e-06  8.32250771e-06  8.36918597e-06
  8.36150111e-06 -1.77915562e-07]
loss:0.42630804425233965


In [41]:
#LS
w,loss = least_squares(y,tX)
print(f'w:{w}\nloss:{loss}')

w:[ 8.03494312e-05 -7.20202273e-03 -6.05417273e-03 -5.47559065e-04
 -1.93874700e-02  4.73451621e-04 -2.60379054e-02  3.25106300e-01
 -3.80780281e-05 -2.72725003e+00 -2.21220140e-01  9.50794091e-02
  6.40351613e-02  2.73550971e+00 -3.31801241e-04 -9.54325119e-04
  2.74026646e+00 -5.34164892e-04  9.73498582e-04  3.69225052e-03
  3.54487449e-04 -5.43344598e-04 -3.30448035e-01 -1.40800498e-03
  8.31432887e-04  1.02117272e-03 -1.68047416e-03 -5.83664817e-03
 -1.11087997e-02  2.72770997e+00]
loss:0.33968680947709307


In [42]:
#RR
lambda_=0.1
w,loss = ridge_regression(y,tX,lambda_)
print(f'w:{w}\nloss:{loss}')

w:[ 8.03357874e-05 -7.20229357e-03 -6.05414322e-03 -5.47524708e-04
 -1.93892567e-02  4.73474160e-04 -2.60360866e-02  3.25109901e-01
 -3.81780647e-05 -4.25488847e-01 -2.21215574e-01  9.50773063e-02
  6.40334364e-02  4.33748519e-01 -3.32367323e-04 -9.54270952e-04
  4.38505213e-01 -5.32787121e-04  9.72271361e-04  3.69233071e-03
  3.55493260e-04 -5.43272790e-04 -3.30447361e-01 -1.40802204e-03
  8.31788674e-04  1.02083858e-03 -1.68036824e-03 -5.83753485e-03
 -1.11063383e-02  4.25948612e-01]
loss:0.3396876032273416


In [ ]:
#LR
initial_w=np.zeros(tX.shape[1])
max_iters=1000
gamma=1e-6
w,loss = logistic_regression(y,tX,initial_w,max_iters,gamma)
print(f'w:{w}\nloss:{loss}')

In [70]:
#RLR
initial_w=np.zeros(tX.shape[1])
max_iters=300
gamma=1e-9
lambda_=0.01
w,loss = reg_logistic_regression(y,tX, lambda_, initial_w, max_iters,gamma)
print(f'w:{w}\nloss:{loss}')

w:[ 2.26600697e-02 -2.68746639e-02  1.56197477e-02  9.33999980e-03
  8.90385311e-03  8.45174582e-03  8.82654311e-03  1.60471727e-03
 -1.44673436e-03  1.22314172e-02 -1.77522520e-02  1.84109276e-02
  8.88260276e-03  1.91508573e-02 -7.87004301e-05 -3.10086187e-04
 -3.14250044e-03  8.37097378e-05  2.89218577e-04 -6.66335362e-03
  5.11345113e-04  1.08159822e-02  8.24004235e-03  1.00867162e-02
  9.60159237e-03  9.60150377e-03  8.83911082e-03  8.87686082e-03
  8.87621516e-03  1.14808594e-02]
loss:160123.40801982983


### Cleaning dataset

#### Transform [-1,1] into [0,1]

In [74]:
#tX[:,:][tX[:,:] == -999] = 0
#we could normalize the data ranging from [0,1] since its binary prediction",
y = (y-min(y))/(max(y)-min(y))


In [75]:
q1 = []
q2 = []
k = 1
for i in range(0,tX.shape[1]):
    q1 = np.percentile(tX[:,i],25)
    q2 = np.percentile(tX[:,i],50)
    q3 = np.percentile(tX[:,i],75)
    tX[:,i][(tX[:,i] < q1 - k*(q3-q1))] = q2
    tX[:,i][(tX[:,i] > q3 + k*(q3-q1))] = q2


In [76]:
tX,_,_ = standardize(tX)
    
#tX[:,:][tX[:,:] = -999].shape[0]

[2.53258940e+01 3.09070463e+01 1.92769071e+01 3.49822486e+01
 4.54479656e+02 5.96812329e+02 4.53018971e+02 7.20517019e-01
 1.34156963e+01 6.58632183e+01 5.31690340e-01 1.19358245e+00
 4.53595814e+02 1.08664430e+01 1.21407622e+00 1.81675941e+00
 1.13794988e+01 1.26497962e+00 1.81660763e+00 1.75168603e+01
 1.81221908e+00 8.15281734e+01 9.77424351e-01 5.32961723e+02
 4.89337307e+02 4.89332905e+02 4.79874536e+02 4.53383717e+02
 4.53388110e+02 5.73167416e+01]


## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '../data/test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)